# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [1]:
from collections import defaultdict
import spacy
from spacy.lang.fr.examples import sentences
!python -m spacy download fr_core_news_md

     ---------------------------------------- 0.0/45.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/45.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/45.8 MB ? eta -:--:--
     ---------------------------------------- 0.3/45.8 MB ? eta -:--:--
     --------------------------------------- 0.5/45.8 MB 796.8 kB/s eta 0:00:57
     --------------------------------------- 0.5/45.8 MB 796.8 kB/s eta 0:00:57
     - -------------------------------------- 1.3/45.8 MB 1.6 MB/s eta 0:00:28
     - -------------------------------------- 1.6/45.8 MB 1.4 MB/s eta 0:00:33
     - -------------------------------------- 1.6/45.8 MB 1.4 MB/s eta 0:00:33
     - -------------------------------------- 1.8/45.8 MB 1.2 MB/s eta 0:00:36
     - -------------------------------------- 1.8/45.8 MB 1.2 MB/s eta 0:00:36
     - -------------------------------------- 1.8/45.8 MB 1.2 MB/s eta 0:00:36
     - ------------------------------------- 2.1/45.8 MB 910.2 kB/s eta 0:00:4


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

nlp = spacy.load('fr_core_news_md')

In [3]:
import re
# Charger le texte nettoyé
with open('../../data/tmp/1955.txt', 'r', encoding='utf-8') as f:
    text_1955 = f.read()
# Travailler sur un extrait de 30 000 caractères à partir d'une position donnée
start = 200000   # position de départ (à ajuster si tu veux une autre portion)
sample = text_1955[start:start+30000]



## Exemple sur un corpus de 1955

In [4]:
# Imprimer les phrases du corpus (1955)
doc = nlp(sample)
sentences = [sent.text for sent in doc.sents]
for s in sentences[:35]:
    print(s)

, à demi-Diogène, en haillons, pieds nus, souriant, sous ses lunettes, d’un sourire hideux, ayant des jambes de squelette, des mains osseuses, gouailleur et philosophe de bas étage, contemplant les misères environnantes et se moquant d’elles.
» Gavarni, poursuit l’un de ses biographes, en fit un type saisissant. «
Vireloque », en face d’un chiffonnier, tombé dans le ruisseau, ahuri par le vin
, laisse échapper ces seules paroles ;
« Sa Majesté le Roi des Animaux !
»... Non, Gavarni, au fond, n’est plus.
« Vireloque » a pris sa place.
Ce qui s’est exactement ’ passé dans l’existence de cet homme, personne ne l’a jamais su.
En soulevant un peu le voile, on apprit simplement que l’artiste était marié, qu’il avait deux enfants et qu’un jour le malheur s’installa à son foyer, pour ne plus le quitter.
Le cadet de ses fils atteint d’un mal mystérieux mourut dans ses bras et le père infortuné demeura inconsolable.
L’idée qu’il avait raté sa vie s’imposa de plus en plus à son esprit, et il se m

In [5]:
# Isoler la quinzième phrase
sent = sentences[14]
sent

'cette consolation lui fut rapidement enlevée, car l’on expropria son domaine et ce fut la ruine définitive.'

In [6]:
# Traiter la phrase avec Spacy
doc = nlp(sample)

In [7]:
type(doc)

spacy.tokens.doc.Doc

In [8]:
doc.text

', à demi-Diogène, en haillons, pieds nus, souriant, sous ses lunettes, d’un sourire hideux, ayant des jambes de squelette, des mains osseuses, gouailleur et philosophe de bas étage, contemplant les misères environnantes et se moquant d’elles. » Gavarni, poursuit l’un de ses biographes, en fit un type saisissant. « Vireloque », en face d’un chiffonnier, tombé dans le ruisseau, ahuri par le vin, laisse échapper ces seules paroles ; « Sa Majesté le Roi des Animaux ! »... Non, Gavarni, au fond, n’est plus. « Vireloque » a pris sa place. Ce qui s’est exactement ’ passé dans l’existence de cet homme, personne ne l’a jamais su. En soulevant un peu le voile, on apprit simplement que l’artiste était marié, qu’il avait deux enfants et qu’un jour le malheur s’installa à son foyer, pour ne plus le quitter. Le cadet de ses fils atteint d’un mal mystérieux mourut dans ses bras et le père infortuné demeura inconsolable. L’idée qu’il avait raté sa vie s’imposa de plus en plus à son esprit, et il se m

In [9]:
doc.to_json()

{'text': ', à demi-Diogène, en haillons, pieds nus, souriant, sous ses lunettes, d’un sourire hideux, ayant des jambes de squelette, des mains osseuses, gouailleur et philosophe de bas étage, contemplant les misères environnantes et se moquant d’elles. » Gavarni, poursuit l’un de ses biographes, en fit un type saisissant. « Vireloque », en face d’un chiffonnier, tombé dans le ruisseau, ahuri par le vin, laisse échapper ces seules paroles ; « Sa Majesté le Roi des Animaux ! »... Non, Gavarni, au fond, n’est plus. « Vireloque » a pris sa place. Ce qui s’est exactement ’ passé dans l’existence de cet homme, personne ne l’a jamais su. En soulevant un peu le voile, on apprit simplement que l’artiste était marié, qu’il avait deux enfants et qu’un jour le malheur s’installa à son foyer, pour ne plus le quitter. Le cadet de ses fils atteint d’un mal mystérieux mourut dans ses bras et le père infortuné demeura inconsolable. L’idée qu’il avait raté sa vie s’imposa de plus en plus à son esprit, e

In [10]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

', à demi-Diogène, en haillons, pieds nus, souriant, sous ses lunettes, d’un sourire hideux, ayant des jambes de squelette, des mains osseuses, gouailleur et philosophe de bas étage, contemplant les misères environnantes et se moquant d’elles.' contient les entités suivantes : demi-Diogène (PER)
'» Gavarni, poursuit l’un de ses biographes, en fit un type saisissant. «' contient les entités suivantes : Gavarni (LOC)
'Vireloque », en face d’un chiffonnier, tombé dans le ruisseau, ahuri par le vin' contient les entités suivantes : Vireloque (LOC)
', laisse échapper ces seules paroles ;' ne contient aucune entité
'« Sa Majesté le Roi des Animaux !' contient les entités suivantes : Majesté le Roi des Animaux (PER)
'»... Non, Gavarni, au fond, n’est plus.' contient les entités suivantes : Gavarni (LOC)
'« Vireloque » a pris sa place.' contient les entités suivantes : Vireloque (LOC)
'Ce qui s’est exactement ’ passé dans l’existence de cet homme, personne ne l’a jamais su.' ne contient aucune

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [11]:
# Charger le texte
n=1000000
text = open("../../data/tmp/1955.txt", encoding='utf-8').read()[:n]

In [12]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: total: 1min 30s
Wall time: 2min 48s


In [13]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [14]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Rossel apparait 219 fois dans le corpus
Ecrire Ag apparait 22 fois dans le corpus
trav apparait 21 fois dans le corpus
Napoléon apparait 19 fois dans le corpus
- Tél apparait 17 fois dans le corpus
curric apparait 17 fois dans le corpus
atel apparait 13 fois dans le corpus
culs apparait 13 fois dans le corpus
curr apparait 13 fois dans le corpus
Inut apparait 13 fois dans le corpus
M. Collard apparait 13 fois dans le corpus
M. Mendès-France apparait 11 fois dans le corpus
Rossel num apparait 11 fois dans le corpus
M. Smelten apparait 11 fois dans le corpus
Sir Winston Churchill apparait 10 fois dans le corpus
A.F.P. apparait 10 fois dans le corpus
Mutuel apparait 9 fois dans le corpus
Reine apparait 9 fois dans le corpus
Louise apparait 9 fois dans le corpus
Créd apparait 9 fois dans le corpus
F. C. apparait 8 fois dans le corpus
chamb apparait 8 fois dans le corpus
Agence Rossel apparait 8 fois dans le corpus
Dagobert apparait 8 fois dans le corpus
Houot apparait 8 fois dans le corpus

liste des lieux (LOC) et les organisations (ORG) et personne (PER) les plus mentionnées dans le corpus

In [15]:
# Diviser le texte en segments pour éviter les limites de taille
from collections import Counter

segment_length = 500000
segments = [text_1955[i:i+segment_length] for i in range(0, len(text_1955), segment_length)]

entities = []

for seg in segments:
    doc = nlp(seg)
    # ajouter les entités extraites dans la liste globale
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))

locations = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
organizations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
persons = [ent.text for ent in doc.ents if ent.label_ == "PER"]

# --- Compter les occurrences ---
loc_counts = Counter(locations)
org_counts = Counter(organizations)
per_counts = Counter(persons)

# --- Afficher les 10 plus fréquentes ---
print("\n Lieux les plus mentionnés :")
print(loc_counts.most_common(10))

print("\n Organisations les plus mentionnées :")
print(org_counts.most_common(10))

print("\n Personnes les plus mentionnées :")
print(per_counts.most_common(10))


 Lieux les plus mentionnés :
[('fr', 58), ('Bruxelles', 47), ('A', 43), ('I', 30), ('Anvers', 25), ('Belgique', 22), ('Brux', 21), ('Paris', 19), ('F', 17), ('Fr', 16)]

 Organisations les plus mentionnées :
[('Agence Rossel', 8), ('bur', 7), ('ch', 6), ('Agence Rossel num', 6), ('PROVINCIALE', 5), ('Royal Dutch', 5), ('dem', 4), ('PATISS', 4), ('trav', 3), ('EMPLOYEE', 3)]

 Personnes les plus mentionnées :
[('Ag', 90), ('Rossel', 62), ('M', 16), ('pr', 10), ('mod', 8), ('Mikaël', 8), ('curric', 7), ('Louise', 7), ('rem', 6), ('Jette', 6)]
